In [3]:
import gurobipy as gp
from gurobipy import GRB

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random as rd
from datetime import datetime, timedelta

import math

#  Matching Algorithm

In [5]:
now = datetime.now()
a = rd.randrange(60*17, 60*19, 5)
print(a)

hours = timedelta(minutes=a).seconds // 3600
minutes = (timedelta(minutes=a).seconds // 60) % 60
datetime(now.year, now.month, now.day, hour=hours, minute=minutes).strftime('%Y-%m-%d %H:%M:%S')

1060


'2023-03-09 17:40:00'

In [6]:
now = datetime.now()
num_user = 50

# 출발 희망 시각[변환 전]
start = [ rd.randrange(60*17, 60*19, 5) for i in range(num_user)]
# 출발 희망 시각[변환 후]
start_time = [ datetime(now.year, now.month, now.day, hour=timedelta(minutes=a).seconds // 3600, minute=(timedelta(minutes=a).seconds // 60) % 60).strftime('%Y-%m-%d %H:%M:%S') for a in start]

# 대기 가능 시간
wait = [ rd.randrange(10, 60,10) for i in range(num_user)]

# 최대 대기 시각[변환 전]
tolerance = [start[i] + wait[i] for i in range(num_user)]
# 최대 대기기 시각[변환 후]
tolerance_time = [ datetime(now.year, now.month, now.day, hour=timedelta(minutes=a).seconds // 3600, minute=(timedelta(minutes=a).seconds // 60) % 60).strftime('%Y-%m-%d %H:%M:%S') for a in tolerance]

# 우선순위
priority = [ i for i in range(1,num_user+1)]

# range_time
range_time = [(start[i], tolerance[i]) for i in range(num_user)]

# ID
ID = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','AA','BB','CC','DD','EE','FF','GG','HH','II','JJ','KK','LL','MM','NN','OO','PP','QQ','RR','SS','TT','UU','VV','WW','XX','YY','ZZ']
id = ID[:num_user]

# 출발지 lat(위도)&lon(경도) [0,1000 사이의 난수] '도'가 아니라 가시를 위해 '초'로 표시하였다.
# start_lat = [round(1000*rd.random(),4) for i in range(num_user)]
# start_lon = [round(1000*rd.random(),4) for i in range(num_user)]
start_lat = [rd.randrange(100, 400, 100) for i in range(num_user)]
start_lon = [rd.randrange(100, 400, 100) for i in range(num_user)]
start_loc = [(start_lat[i], start_lon[i]) for i in range(num_user)]

# 출발지 lat(위도)&lon(경도) [0,1000 사이의 난수]
des_lat = [round(1000*rd.random(),4) for i in range(num_user)]
des_lon = [round(1000*rd.random(),4) for i in range(num_user)]
des_loc = [(des_lat[i], des_lon[i]) for i in range(num_user)]

Step1

Step2  $A$의 출발지와 목적지를 직선으로 이은 벡터를 $v_A$라고 하고, 그 외의 $i$의 출발지와 목적지를 직선으로 이은 벡터를 $v_i$라고 하자

In [7]:
df1 = pd.DataFrame(dict(zip(['ID','출발희망시각','최대대기시각','우선순위','시간범위','출발위치','도착위치'],[id, start_time, tolerance_time, priority, range_time, start_loc, des_loc])))
df1 = df1.sort_values(by=['출발희망시각','최대대기시각']) # sorting
df1 = df1.set_index('우선순위')
df1

,ID,출발희망시각,최대대기시각,시간범위,출발위치,도착위치
우선순위,,,,,,
27,AA,2023-03-09 17:00:00,2023-03-09 17:10:00,"(1020, 1030)","(200, 100)","(540.2529, 552.244)"
40,NN,2023-03-09 17:00:00,2023-03-09 17:50:00,"(1020, 1070)","(100, 300)","(278.4532, 8.526)"
4,D,2023-03-09 17:05:00,2023-03-09 17:25:00,"(1025, 1045)","(100, 200)","(412.8099, 355.0749)"
9,I,2023-03-09 17:10:00,2023-03-09 17:20:00,"(1030, 1040)","(100, 200)","(498.3109, 401.6471)"
30,DD,2023-03-09 17:10:00,2023-03-09 17:40:00,"(1030, 1060)","(200, 200)","(428.1948, 926.0876)"
23,W,2023-03-09 17:10:00,2023-03-09 17:50:00,"(1030, 1070)","(300, 100)","(148.4867, 300.3196)"
21,U,2023-03-09 17:10:00,2023-03-09 18:00:00,"(1030, 1080)","(300, 100)","(220.6614, 917.7692)"
12,L,2023-03-09 17:15:00,2023-03-09 17:35:00,"(1035, 1055)","(100, 200)","(573.0015, 383.4474)"
41,OO,2023-03-09 17:15:00,2023-03-09 17:35:00,"(1035, 1055)","(100, 100)","(519.1966, 625.5384)"


Step3. 1순위($A$)의 출발희망시각과 최대 대기 시각과 겹치지 않는 ID는 탈락한다.

In [8]:
start, end = df1.loc[1]['시간범위']
for index, row in df1.iterrows():
  st, en = df1.loc[index]['시간범위']
  if start>=en or end<=st:
    df1 = df1.drop(index, axis=0)

Step4. $A$의 출발지&도착지 기준 반경 $m$km 이내에 벡터가 겹치는 ID만 남긴다

In [9]:
# A의 위치정보
start_lat, start_lon = df1.loc[1]['출발위치']
end_lat, end_lon = df1.loc[1]['도착위치']

m = 20 # 20초
m = m*1000/37 # km를 초로 치환, 1초는 37m이다

# 직선의 방정식
def line_eq(x1, y1, x2, y2):
  m = (y2 - y1) / (x2 - x1)
  b = y1 - m * x1
  A, B, C = -m, 1, -b
  return A, B, C

# 직선과 A 사이의 거리
def distance(x1, y1, a, b, c):
  dist = abs(a*x1 + b*y1 + c) / math.sqrt(a*a + b*b)
  return dist

for index, row in df1.iterrows():
  st_lat, st_lon = df1.loc[index]['출발위치']
  en_lat, en_lon = df1.loc[index]['도착위치']

  # 각 ID의 벡터
  a, b, c = line_eq(st_lat, st_lon, en_lat, en_lon)
  # 각 ID의 벡터와 A 출발점 사이의 거리
  dist_from_st = distance(start_lat, start_lon, a, b, c)
  # 각 ID의 벡터와 B 출발점 사이의 거리
  dist_from_en = distance(end_lat, end_lon, a, b, c)

  # print(dist_from_st, dist_from_en)
  
  if dist_from_st >= m and dist_from_en >= m:
    df1 = df1.drop(index, axis=0)

df1

,ID,출발희망시각,최대대기시각,시간범위,출발위치,도착위치
우선순위,,,,,,
21,U,2023-03-09 17:10:00,2023-03-09 18:00:00,"(1030, 1080)","(300, 100)","(220.6614, 917.7692)"
32,FF,2023-03-09 17:20:00,2023-03-09 18:00:00,"(1040, 1080)","(200, 200)","(417.3702, 95.6003)"
11,K,2023-03-09 17:20:00,2023-03-09 18:10:00,"(1040, 1090)","(300, 300)","(440.3613, 627.4039)"
19,S,2023-03-09 17:20:00,2023-03-09 18:10:00,"(1040, 1090)","(300, 300)","(985.3178, 68.0464)"
18,R,2023-03-09 17:30:00,2023-03-09 18:10:00,"(1050, 1090)","(100, 100)","(810.1204, 46.3672)"
35,II,2023-03-09 17:35:00,2023-03-09 18:25:00,"(1055, 1105)","(300, 300)","(510.6593, 13.5029)"
2,B,2023-03-09 17:40:00,2023-03-09 18:00:00,"(1060, 1080)","(300, 300)","(316.8967, 33.0356)"
14,N,2023-03-09 17:40:00,2023-03-09 18:10:00,"(1060, 1090)","(100, 300)","(123.4641, 986.0844)"
26,Z,2023-03-09 17:40:00,2023-03-09 18:30:00,"(1060, 1110)","(300, 200)","(430.9134, 314.3691)"


In [10]:
# m = 10
for index, row in df1.iterrows():
  st_lat, st_lon = df1.loc[index]['출발위치']
  en_lat, en_lon = df1.loc[index]['도착위치']
  a, b, c = line_eq(st_lat, st_lon, en_lat, en_lon)
  dist_from_st = distance(start_lat, start_lon, a, b, c)
  dist_from_en = distance(end_lat, end_lon, a, b, c)
  print(min(dist_from_st, dist_from_en))

89.87617567523057
0.0
52.507210445432186
126.78120479692105
107.24718843179265
139.80413025493016
106.11686309199618
103.35957282051157
65.79175425449887
53.258119750676286
38.99959029837082
0.0
107.10597089789026
6.953511623681656
0.0
83.13928652928978
55.23618423782769


Step5. 벡터 연산을 통해 구한 사이각과 스칼라값을 통해 조건을 만족하지 않는 ID는 탈락한다.

In [11]:
# A의 위치정보
start_lat, start_lon = df1.loc[1]['출발위치']
end_lat, end_lon = df1.loc[1]['도착위치']

a_vec = [start_lat-end_lat, start_lon-end_lon]
print(a_vec)
for index, row in df1.iterrows():
  st_lat, st_lon = df1.loc[index]['출발위치']
  en_lat, en_lon = df1.loc[index]['도착위치']
  i_vec = [st_lat-en_lat, st_lon-en_lon]

  if max(math.sqrt(a_vec[0]**2 + a_vec[1]**2),math.sqrt(i_vec[0]**2 + i_vec[1]**2)) < math.sqrt((a_vec[0]-i_vec[0])**2+(a_vec[1]-i_vec[1])**2):
    df1 = df1.drop(index, axis=0)

df2 = df1.copy()
df1

[-401.25660000000005, -155.16750000000002]


,ID,출발희망시각,최대대기시각,시간범위,출발위치,도착위치
우선순위,,,,,,
21,U,2023-03-09 17:10:00,2023-03-09 18:00:00,"(1030, 1080)","(300, 100)","(220.6614, 917.7692)"
32,FF,2023-03-09 17:20:00,2023-03-09 18:00:00,"(1040, 1080)","(200, 200)","(417.3702, 95.6003)"
11,K,2023-03-09 17:20:00,2023-03-09 18:10:00,"(1040, 1090)","(300, 300)","(440.3613, 627.4039)"
19,S,2023-03-09 17:20:00,2023-03-09 18:10:00,"(1040, 1090)","(300, 300)","(985.3178, 68.0464)"
18,R,2023-03-09 17:30:00,2023-03-09 18:10:00,"(1050, 1090)","(100, 100)","(810.1204, 46.3672)"
14,N,2023-03-09 17:40:00,2023-03-09 18:10:00,"(1060, 1090)","(100, 300)","(123.4641, 986.0844)"
26,Z,2023-03-09 17:40:00,2023-03-09 18:30:00,"(1060, 1110)","(300, 200)","(430.9134, 314.3691)"
16,P,2023-03-09 17:45:00,2023-03-09 18:05:00,"(1065, 1085)","(300, 200)","(615.5193, 1.4605)"
38,LL,2023-03-09 17:45:00,2023-03-09 18:35:00,"(1065, 1115)","(300, 300)","(534.0102, 722.3017)"


In [12]:
for index, row in df1.iterrows():
  st_lat, st_lon = df1.loc[index]['출발위치']
  en_lat, en_lon = df1.loc[index]['도착위치']
  i_vec = [st_lat-en_lat, st_lon-en_lon]
  print(i_vec)

[79.33860000000001, -817.7692]
[-217.3702, 104.3997]
[-140.36130000000003, -327.4039]
[-685.3178, 231.9536]
[-710.1204, 53.6328]
[-23.464100000000002, -686.0844]
[-130.91340000000002, -114.3691]
[-315.51930000000004, 198.5395]
[-234.01020000000005, -422.3017]
[-657.0861, -48.616000000000014]
[-326.45550000000003, 22.755200000000002]
[-401.25660000000005, -155.16750000000002]
[-553.2953, -835.0132]


Step6. (추후 개발)

Step7. $A$와 선정된 후보들을 경유했을 때의 실제 경로의 운행 시간과 비용을 바탕으로 최적의 경유지를 탐색한다

In [13]:
start_lat, start_lon = df1.loc[1]['출발위치']
end_lat, end_lon = df1.loc[1]['도착위치']

df1 = df1.drop(1, axis=0)
df1

,ID,출발희망시각,최대대기시각,시간범위,출발위치,도착위치
우선순위,,,,,,
21,U,2023-03-09 17:10:00,2023-03-09 18:00:00,"(1030, 1080)","(300, 100)","(220.6614, 917.7692)"
32,FF,2023-03-09 17:20:00,2023-03-09 18:00:00,"(1040, 1080)","(200, 200)","(417.3702, 95.6003)"
11,K,2023-03-09 17:20:00,2023-03-09 18:10:00,"(1040, 1090)","(300, 300)","(440.3613, 627.4039)"
19,S,2023-03-09 17:20:00,2023-03-09 18:10:00,"(1040, 1090)","(300, 300)","(985.3178, 68.0464)"
18,R,2023-03-09 17:30:00,2023-03-09 18:10:00,"(1050, 1090)","(100, 100)","(810.1204, 46.3672)"
14,N,2023-03-09 17:40:00,2023-03-09 18:10:00,"(1060, 1090)","(100, 300)","(123.4641, 986.0844)"
26,Z,2023-03-09 17:40:00,2023-03-09 18:30:00,"(1060, 1110)","(300, 200)","(430.9134, 314.3691)"
16,P,2023-03-09 17:45:00,2023-03-09 18:05:00,"(1065, 1085)","(300, 200)","(615.5193, 1.4605)"
38,LL,2023-03-09 17:45:00,2023-03-09 18:35:00,"(1065, 1115)","(300, 300)","(534.0102, 722.3017)"


In [14]:
df3 = df1.copy()
epoch = 4
'''
우선 x좌표를 기준으로 sorting하여 경유지를 설정하였다. 추후 경유지API 가져와 변경 예정
[2021년 서울 차량 통행속도 분석]에 따르면 지난해 도심 통행속도는 시속 23㎞'라는 정보를 활용해 시간을 산출하였다.
1초 사이의 거리는 0.037 km = 37 m 이므로 이를 기반으로 거리를 산출하였다.
'''

def distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    dist = math.sqrt(dx*dx + dy*dy)
    return dist

for index, row in df1.iterrows():
  st_lat, st_lon = start_lat, start_lon
  en_lat, en_lon = df1.loc[index]['도착위치']
  # lat_lon = {
  #     start_lat: start_lon,
  #     end_lat: end_lon,
  #     st_lat: st_lon,
  #     en_lat: en_lon
  #     }
  lat_lon = {
    'a_start': df1.loc[index]['출발위치'],
    'a_end': df1.loc[index]['도착위치'],
    'i_start': (start_lat, start_lon),
    'i_end': (end_lat, end_lon)
      }
  # 원점에서 떨어진 거리순으로 정렬
  dis_lat_lon = {
      distance(0, 0, start_lat, start_lon): 'a_start', # a 시작지점
      distance(0, 0, end_lat, end_lon): 'a_end', # a 끝지점
      distance(0, 0, st_lat, st_lon): 'i_start', # i 시작지점
      distance(0, 0, en_lat, en_lon): 'i_end' # i 끝지점
      }


  dis = sorted([distance(0, 0, start_lat, start_lon), distance(0, 0, end_lat,end_lon), distance(0, 0, st_lat, st_lon), distance(0, 0, en_lat, en_lon)])
  print(dis)
  routing_lat_lon = [ dis_lat_lon[key] for key in dis]
  print(routing_lat_lon)
  print([ lat_lon[key] for key in routing_lat_lon])

  routing_lat = [lat for lat,lon in [ lat_lon[key] for key in routing_lat_lon]]
  routing_lon = [lon for lat,lon in [ lat_lon[key] for key in routing_lat_lon]]

  print('routing_lat: ',routing_lat)
  print('routing_lon: ',routing_lon)
  print(f'A start: {start_lat, start_lon} A end: {end_lat, end_lon}')
  print(f'I start: {st_lat, st_lon} I end: {en_lat, en_lon}')

  # 경유 node
  if routing_lat.index(start_lat) > routing_lat.index(end_lat) or routing_lat.index(st_lat) > routing_lat.index(en_lat):
    routing_lat = routing_lat[::-1]
    routing_lon = routing_lon[::-1]
    a_routing_idx = [routing_lat.index(start_lat), routing_lat.index(end_lat)]
    i_routing_idx = [routing_lat.index(st_lat), routing_lat.index(en_lat)]


  else:
    a_routing_idx = [routing_lat.index(start_lat), routing_lat.index(end_lat)]
    i_routing_idx = [routing_lat.index(st_lat), routing_lat.index(en_lat)]


  a_route_dist = 0
  a_dist = 0
  start_point = a_routing_idx[0]
  end_point = a_routing_idx[1]
  print(start_point<end_point)


  # 0>1 , 2>3 인 경우 제외
  if (start_point, end_point) == (0,1) or (start_point, end_point) == (2,3):
    df1 = df1.drop(index, axis=0)
    print('wrong match')
    print()

  else:
    print(start_point, end_point)

    # 타킷경유시간(분)
    point = start_point
    try:
      for i in range(epoch):
        # a가 노드
        x1 = routing_lat[point]
        y1 = routing_lon[point]
        # 다음 노드
        point += 1

        x2 = routing_lat[point]
        y2 = routing_lon[point]
        dist_btn_node = distance(x1, y1, x2, y2) * 0.037 # 초(각도) >> km(거리)
        a_route_dist += dist_btn_node
        if point==end_point:
          break
    except:
      pass

    a_route_time = 60/23 * a_route_dist # km(거리) >> 분(시간)
    df1.loc[index,'타깃경유시간(분)'] = a_route_time
    df1.loc[index,'타깃경유거리(km)'] = a_route_dist

    # 타킷원래시간(분)
    # a 출발
    x1 = routing_lat[start_point]
    y1 = routing_lon[start_point]
    # a 도착
    x2 = routing_lat[end_point]
    y2 = routing_lon[end_point]
    a_dist = distance(x1, y1, x2, y2) * 0.037
    a_time = 60/23 * a_dist
    df1.loc[index,'타깃원래시간(분)'] = a_time
    df1.loc[index,'타깃원래거리(km)'] = a_dist


    i_route_dist = 0
    i_dist = 0
    start_point = i_routing_idx[0]
    end_point = i_routing_idx[1]
    print(start_point, end_point)

    # 경유시간(분)
    point = start_point
    try:
      for i in range(epoch):
        # a가 노드
        x1 = routing_lat[point]
        y1 = routing_lon[point]
        # 다음 노드
        point += 1
        x2 = routing_lat[point]
        y2 = routing_lon[point]
        dist_btn_node = distance(x1, y1, x2, y2) * 0.037 # 초(각도) >> km(거리)
        i_route_dist += dist_btn_node
        if point==end_point:
          break
    except:
      pass

    i_route_time = 60/23 * i_route_dist # km(거리) >> 분(시간)
    df1.loc[index,'경유시간(분)'] = i_route_time
    df1.loc[index,'경유거리(km)'] = i_route_dist

    # 원래시간(분)
    # i 출발
    x1 = routing_lat[start_point]
    y1 = routing_lon[start_point]
    # i 도착
    x2 = routing_lat[end_point]
    y2 = routing_lon[end_point]
    i_dist = distance(x1, y1, x2, y2) * 0.037
    i_time = 60/23 * i_dist
    df1.loc[index,'원래시간(분)'] = i_time
    df1.loc[index,'원래거리(km)'] = i_dist

    print(routing_lat, routing_lon)
    print()


    total_route_dist = 0
    point = 0

    # 전체체경유시간(분)
    try:
      for i in range(epoch):
        # a가 노드
        x1 = routing_lat[point]
        y1 = routing_lon[point]
        # 다음 노드
        point += 1

        x2 = routing_lat[point]
        y2 = routing_lon[point]
        dist_btn_node = distance(x1, y1, x2, y2) * 0.037 # 초(각도) >> km(거리)
        total_route_dist += dist_btn_node
        if point==4:
          break
    except:
      pass

    total_time = 60/23 * total_route_dist
    df1.loc[index,'총이동시간(분)'] = total_time
    df1.loc[index,'총경유거리(km)'] = total_route_dist

df1

[282.842712474619, 282.842712474619, 698.321882873371, 943.9235975006663]
['i_start', 'i_start', 'a_end', 'i_end']
[(200, 200), (200, 200), (220.6614, 917.7692), (601.2566, 355.1675)]
routing_lat:  [200, 200, 220.6614, 601.2566]
routing_lon:  [200, 200, 917.7692, 355.1675]
A start: (200, 200) A end: (601.2566, 355.1675)
I start: (200, 200) I end: (220.6614, 917.7692)
True
0 3
0 2
[200, 200, 220.6614, 601.2566] [200, 200, 917.7692, 355.1675]

[282.842712474619, 282.842712474619, 428.17905274327705, 698.321882873371]
['i_start', 'i_start', 'i_end', 'a_end']
[(200, 200), (200, 200), (601.2566, 355.1675), (417.3702, 95.6003)]
routing_lat:  [200, 200, 601.2566, 417.3702]
routing_lon:  [200, 200, 355.1675, 95.6003]
A start: (200, 200) A end: (601.2566, 355.1675)
I start: (200, 200) I end: (417.3702, 95.6003)
True
0 2
0 3
[200, 200, 601.2566, 417.3702] [200, 200, 355.1675, 95.6003]

[282.842712474619, 282.842712474619, 698.321882873371, 766.5205334972443]
['i_start', 'i_start', 'a_end', 'i_en

,ID,출발희망시각,최대대기시각,시간범위,출발위치,도착위치,타깃경유시간(분),타깃경유거리(km),타깃원래시간(분),타깃원래거리(km),경유시간(분),경유거리(km),원래시간(분),원래거리(km),총이동시간(분),총경유거리(km)
우선순위,,,,,,,,,,,,,,,,
21,U,2023-03-09 17:10:00,2023-03-09 18:00:00,"(1030, 1080)","(300, 100)","(220.6614, 917.7692)",134.870910,51.700515,41.524973,15.917906,69.309029,26.568461,69.309029,26.568461,134.870910,51.700515
32,FF,2023-03-09 17:20:00,2023-03-09 18:00:00,"(1040, 1080)","(200, 200)","(417.3702, 95.6003)",41.524973,15.917906,41.524973,15.917906,72.228800,27.687707,23.275373,8.922226,72.228800,27.687707
11,K,2023-03-09 17:20:00,2023-03-09 18:10:00,"(1040, 1090)","(300, 300)","(440.3613, 627.4039)",77.852727,29.843545,41.524973,15.917906,47.329880,18.143121,47.329880,18.143121,77.852727,29.843545
19,S,2023-03-09 17:20:00,2023-03-09 18:10:00,"(1040, 1090)","(300, 300)","(985.3178, 68.0464)",123.147129,47.206399,41.524973,15.917906,76.862813,29.464078,76.862813,29.464078,123.147129,47.206399
18,R,2023-03-09 17:30:00,2023-03-09 18:10:00,"(1050, 1090)","(100, 100)","(810.1204, 46.3672)",96.711750,37.072837,41.524973,15.917906,60.728203,23.279144,60.728203,23.279144,96.711750,37.072837
14,N,2023-03-09 17:40:00,2023-03-09 18:10:00,"(1060, 1090)","(100, 300)","(123.4641, 986.0844)",152.622018,58.505107,41.524973,15.917906,76.233015,29.222656,76.233015,29.222656,152.622018,58.505107
26,Z,2023-03-09 17:40:00,2023-03-09 18:30:00,"(1060, 1110)","(300, 200)","(430.9134, 314.3691)",41.524973,15.917906,41.524973,15.917906,58.431800,22.398857,24.872154,9.534326,58.431800,22.398857
16,P,2023-03-09 17:45:00,2023-03-09 18:05:00,"(1065, 1085)","(300, 200)","(615.5193, 1.4605)",41.524973,15.917906,41.524973,15.917906,75.693132,29.015701,44.449725,17.039061,75.693132,29.015701
38,LL,2023-03-09 17:45:00,2023-03-09 18:35:00,"(1065, 1115)","(300, 300)","(534.0102, 722.3017)",95.866479,36.748817,41.524973,15.917906,59.840511,22.938863,59.840511,22.938863,95.866479,36.748817


Step7-5. (동승자 지불 금액의 합-택시 지불 금액)/택시 지불 금액 > 0.1 이어야 5% 할인을 보장할 수 있다.

Step8. 최소최대 소요시간 증가 비율에 해당하는 ID(1순위동승예약자)와 최종 매칭을 진행한다

In [ ]:
len(df2)

In [ ]:
ratio_index = {}
df2 = df1.copy()
len_df2 = len(df2)

alpha_min = 0.05 # 동승객 최소 할인율
beta_min = 0.05 # 동승객 최소 추가 할인율
gamma = 0.05 # 혹시 최소 수익률
overlap_ratio = 0.4

for _ in range(len_df2):
  for index, row in df2.iterrows():
    sort_last = []
    print(f'target : {index}')
    i_org = df2.loc[index,'원래시간(분)']
    i_rou = df2.loc[index,'경유시간(분)']
    a_org = df2.loc[index,'타깃원래시간(분)']
    a_rou = df2.loc[index,'타깃경유시간(분)']
    sort_last.append(max((i_rou-i_org)/i_org,(a_rou-a_org)/a_org))
    ratio_index[max((i_rou-i_org)/i_org,(a_rou-a_org)/a_org)] = index

    target_user = df2.loc[ratio_index[min(sort_last)]]['ID']
    target_index = ratio_index[min(sort_last)]

    a_ind_pred = int(4800 + (df2.loc[target_index,'타깃원래거리(km)']-1.6)*100/1.31 + (df2.loc[target_index,'타깃원래시간(분)']-96/23)*200)
    i_ind_pred = int(4800 + (df2.loc[target_index,'원래거리(km)']-1.6)*100/1.31 + (df2.loc[target_index,'원래시간(분)']-96/23)*200)
    driver_rou_pred = int(4800 + (df2.loc[target_index,'총경유거리(km)']-1.6)*100/1.31 + (df2.loc[target_index,'총이동시간(분)']-96/23)*200)

    det = a_ind_pred*(1-alpha_min-beta_min*(a_rou-a_org)/a_org)+ i_ind_pred*(1-alpha_min-beta_min*(i_rou-i_org)/i_org) - driver_rou_pred*(1+gamma)

    # if (a_ind_pred + i_ind_pred - driver_rou_pred)/driver_rou_pred > overlap_ratio:
    if (det > 0):
      print(f"최종적으로 'A'와 '{target_user}'을(를) 매칭해준다. ( {det:.2f} )")
      print()
      break
    else:
      df2 = df2.drop(index, axis=0)
      print(f"{target_user}, : {det:.2f} => 최종 매칭 X")
      print()
  # if (a_ind_pred + if_ind_pred - driver_rou_pred)/driver_rou_pred > overlap_ratio:
  if (det > 0):
    break

In [ ]:
'''
기본요금 1.6km 까지 4800원
131m 당 100원
30초당 100원
'''
target_index = ratio_index[min(sort_last)]
print(target_index)
df1.loc[target_index]

# Settlement Algorithm

In [ ]:
a_ind_pred = int(4800 + (df1.loc[target_index,'타깃원래거리(km)']-1.6)*100/1.31 + (df1.loc[target_index,'타깃원래시간(분)']-96/23)*200)
i_ind_pred = int(4800 + (df1.loc[target_index,'원래거리(km)']-1.6)*100/1.31 + (df1.loc[target_index,'원래시간(분)']-96/23)*200)
driver_rou_pred = int(4800 + (df1.loc[target_index,'총경유거리(km)']-1.6)*100/1.31 + (df1.loc[target_index,'총이동시간(분)']-96/23)*200)

print(a_ind_pred, i_ind_pred, driver_rou_pred)

In [ ]:
a_ind_pred = int(4800 + (df1.loc[target_index,'타깃원래거리(km)']-1.6)*100/1.31 + (df1.loc[target_index,'타깃원래시간(분)']-96/23)*200)
i_ind_pred = int(4800 + (df1.loc[target_index,'원래거리(km)']-1.6)*100/1.31 + (df1.loc[target_index,'원래시간(분)']-96/23)*200)
driver_rou_pred = int(4800 + (df1.loc[target_index,'총경유거리(km)']-1.6)*100/1.31 + (df1.loc[target_index,'총이동시간(분)']-96/23)*200)

print(a_ind_pred, i_ind_pred, driver_rou_pred)

alpha = 0.5 # 동승객 최대 할인율
beta = 0.5 # 동승객 최대 추가 할인율
gamma = 0.05 # 혹시 최소 수익률
eplison = 0.03 # 실제 요금/예상요금 비율

lower_bound = 0.2 # 알파베타 비율 최소 상한선
upper_bound = 0.5 # 알파베타 비율 최대 상한선

addia = 0.001
addib = 0.01

# 최초의 상태
a_beta = beta*((a_rou-a_org)/a_org)
a_payment = int((1-(alpha+a_beta))*a_ind_pred)

i_beta = beta*((i_rou-i_org)/i_org)
i_payment = int((1-(alpha+i_beta))*i_ind_pred)

print(f'최적화 전 objective value: {a_payment + i_payment - driver_rou_pred*(1+gamma):.0f}원')
print()

# 만약 목적식이 < 0 이라면 >0가 된 순간 끝
if a_payment + i_payment - driver_rou_pred*(1+gamma) <0:
  while(True):
    a_rou, a_org = df1.loc[target_index,'타깃경유시간(분)'], df1.loc[target_index,'타깃원래시간(분)']
    i_rou, i_org = df1.loc[target_index,'경유시간(분)'], df1.loc[target_index,'원래시간(분)']

    a_beta = beta*((a_rou-a_org)/a_org)
    a_payment = int((1-(alpha+a_beta))*a_ind_pred)

    i_beta = beta*((i_rou-i_org)/i_org)
    i_payment = int((1-(alpha+i_beta))*i_ind_pred)

    ## 알파베타 비율이 정산범위 내
    if max(a_beta/alpha,i_beta/alpha)>=lower_bound and min(a_beta/alpha,i_beta/alpha)<=upper_bound:
      # alpha, beta 감소
      beta -= addib

      # alpha 감소
      if alpha>0.05:
        alpha -= addia
      else: alpha = alpha
    
    ## 알파베타 비율이 정산범위 보다 작을 때
    elif max(a_beta/alpha,i_beta/alpha)<=lower_bound:
      # alpha감소 & beta 증가
      alpha -= addia
      beta += addib

    ## 알파베타 비율이 정산범위 보다 클 때
    elif max(a_beta/alpha,i_beta/alpha)>=lower_bound:
      # alpha증가 & beta 감소
      alpha += addia
      beta -= addib


    if a_payment + i_payment - driver_rou_pred*(1+gamma)>0:
      print("type1")
      break


# 만약 목적식이 > 0 이라면 <0가 된 순간 전단계 되돌린 후 끝
else:
  while(True):
    a_rou, a_org = df1.loc[target_index,'타깃경유시간(분)'], df1.loc[target_index,'타깃원래시간(분)']
    i_rou, i_org = df1.loc[target_index,'경유시간(분)'], df1.loc[target_index,'원래시간(분)']

    a_beta = beta*((a_rou-a_org)/a_org)
    a_payment = int((1-(alpha+a_beta))*a_ind_pred)

    i_beta = beta*((i_rou-i_org)/i_org)
    i_payment = int((1-(alpha+i_beta))*i_ind_pred)

    ## 알파베타 비율이 정산범위 내
    if max(a_beta/alpha,i_beta/alpha)>=lower_bound and max(a_beta/alpha,i_beta/alpha)<=upper_bound:
      # alpha, beta 감소
      beta -= addib

      # alpha 감소
      if alpha>0.05:
        alpha -= addia
      else: alpha = alpha
    
    ## 알파베타 비율이 정산범위 보다 작을 때
    elif max(a_beta/alpha,i_beta/alpha)<=lower_bound:
      # alpha감소 & beta 증가
      alpha -= addia
      # beta += addib

    ## 알파베타 비율이 정산범위 보다 클 때
    elif max(a_beta/alpha,i_beta/alpha)>=upper_bound:
      # alpha증가 & beta 감소
      alpha += addia
      # beta -= addib

    # 전단계 롤백
    if a_payment + i_payment - driver_rou_pred*(1+gamma)<0:
      if alpha>0.05:
        alpha += addia
      else:
        alpha = alpha

      beta += addib
      a_beta = beta*((a_rou-a_org)/a_org)
      a_payment = int((1-(alpha+a_beta))*a_ind_pred)

      i_beta = beta*((i_rou-i_org)/i_org)
      i_payment = int((1-(alpha+i_beta))*i_ind_pred)
      if a_payment + i_payment - driver_rou_pred*(1+gamma)>0:
        print("type2")
        break
  


print(f'alpha: {alpha:.2f}, beta: {beta:.2f}')
print(f'타깃동승자 ratio: {a_beta/alpha:.2f} / 동승자 ratio: {i_beta/alpha:.2f}')

hoxi_profit_per_matching = round(a_payment + i_payment - (1+eplison)*driver_rou_pred)
print()
print(f'타깃동승자   {alpha*100:.2f}+{a_beta*100:.2f} => {(alpha+a_beta)*100:.2f}% 할인')
print(f'동승자       {alpha*100:.2f}+{i_beta*100:.2f}  => {(alpha+i_beta)*100:.2f}% 할인')
print(f'타깃깃동승자 {a_ind_pred}원 >> {a_payment}원')
print(f'동승자       {i_ind_pred}원 >> {i_payment}원')
print(f'혹시가 얻는 최종 이익 >> {hoxi_profit_per_matching} 원')
print()
print(f'최적화 후 objective value: {a_payment + i_payment - driver_rou_pred*(1+gamma):.0f}원')

gurobi

In [ ]:
C1 = a_ind_pred
C2 = (a_rou-a_org)/a_org
C3 = i_ind_pred
C4 = (i_rou-i_org)/i_org
C5 = driver_rou_pred

LB = 0.2
UB = 0.5

In [ ]:
try:
  # Create a new model
  m = gp.Model('cost_optimization1')

  # Create variables
  alpha = m.addVar(vtype=GRB.CONTINUOUS, name='alpha')
  beta = m.addVar(vtype=GRB.CONTINUOUS, name='beta')
  gamma = 0.05


  # Set objective
  m.setObjective(C1*(1-(alpha+beta*C2))+C3*(1-(alpha+beta*C4))-(1+gamma)*C5, GRB.MINIMIZE)

  # Add constraint
  m.addConstr(max(C2,C4)*beta-UB*alpha <= 0,'const_1')
  m.addConstr(max(C2,C4)*beta-LB*alpha >= 0,'const_2')
  m.addConstr((C1*(1-(alpha+beta*C2))+C3*(1-(alpha+beta*C4))-(1+gamma)*C5) >= 0,'const_3')

  m.addConstr(alpha <= 0.5)
  m.addConstr(alpha >= 0)
  m.addConstr(beta <= 0.5)
  m.addConstr(beta >= 0)
  #m.addConstr(gamma <= 1)

  # Optimize model
  m.optimize()

  for v in m.getVars():
    print('%s : %g' % (v.varName, v.x))
    globals()[v.varName] = v.x
  print('Obj : %g' % m.objVal)

except gp.GurobiError as e:
  print('Error code ' + str(e.errno) + ': ' + str(e))

In [ ]:
print()
print(f'타깃동승자   {alpha*100:.2f}+{beta*C2*100:.2f} => {(alpha+beta*C2)*100:.2f}% 할인')
print(f'동승자       {alpha*100:.2f}+{beta*C4*100:.2f}  => {(alpha+beta*C4)*100:.2f}% 할인')
print(f'타깃깃동승자 {C1}원 >> {(1-alpha+beta*C2)*C1:.0f}원')
print(f'동승자       {C3}원 >> {(1-alpha+beta*C4)*C3:.0f}원')
print(f'혹시가 얻는 최종 이익 >> {(1-alpha+beta*C2)*C1+(1-alpha+beta*C4)*C3-(1+eplison)*C5:.0f} 원')
print()
print(f'최적화 후 objective value: {(1-alpha-beta*C2)*C1+(1-alpha-beta*C4)*C3-(1+gamma)*C5:.0f} 원')